In [17]:
import os
import numpy as np 
import icartt
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import datetime as dt
from datetime import datetime 
# test dataset
dataset = icartt.Dataset("/Users/ischluesche/Documents/Grad_School/chem/MER-TOGA_DC8_20160729_R22.ict")
import pickle 
# Directory containing the ICTs for plotting all
directory = "/Users/C837212720/Desktop/archive_download_27"

import pandas as pd

/Users/ischluesche/miniconda3/envs/chemistry/lib/python3.13/site-packages/icartt/dataset.py:392: UserWarning: Variable short name j23Butanedione_NoProductsSpecified_CAFS does not comply with ICARTT standard v2
  warnings.warn(
/Users/ischluesche/miniconda3/envs/chemistry/lib/python3.13/site-packages/icartt/dataset.py:392: UserWarning: Variable short name C4-C5Alkanes_GMI does not comply with ICARTT standard v2
  warnings.warn(
/Users/ischluesche/miniconda3/envs/chemistry/lib/python3.13/site-packages/icartt/dataset.py:392: UserWarning: Variable short name Days_Since_Land_BdyInf_Most_Recent does not comply with ICARTT standard v2
  warnings.warn(
/Users/ischluesche/miniconda3/envs/chemistry/lib/python3.13/site-packages/icartt/dataset.py:392: UserWarning: Variable short name CO.X does not comply with ICARTT standard v2
  warnings.warn(
/Users/ischluesche/miniconda3/envs/chemistry/lib/python3.13/site-packages/icartt/dataset.py:392: UserWarning: Variable short name Flag.CO.X does not comply

In [18]:
print(dataset.PIName)
print(dataset.missionName)
variable_names = [x for x in dataset.variables]
print(variable_names)
utc_data = dataset.data["UTC_Start"]
print(utc_data)
all_data = dataset.data[:]
print(all_data)

Wofsy,S
ATom
['UTC_Start', 'UTC_Stop_TOGA', 'CFC11_TOGA', 'UTC_Mean_1s', 'P', 'T', 'TAS', 'U', 'V', 'W', 'TEDR', 'REYN', 'G_LAT', 'G_LONG', 'G_ALT', 'POT', 'ROLL', 'HDG', 'PITCH', 'YAW', 'AOA', 'Stoptime_AMSSD', 'MidPoint_AMSSD', 'LAT_AMSSD', 'LON_AMSSD', 'GALT_AMSSD', 'StdToVol_AMSSD', 'OA_PM1_AMSSD', 'SO4_PM1_AMSSD', 'NO3_PM1_AMSSD', 'NH4_PM1_AMSSD', 'Chl_PM1_AMSSD', 'Stoptime_AMS', 'MidPointTime_AMS', 'LAT_AMS', 'LON_AMS', 'ALT_AMS', 'OA_PM1_AMS', 'OA_prec_PM1_AMS', 'OA_DL_PM1_AMS', 'Sulfate_PM1_AMS', 'Sulfate_prec_PM1_AMS', 'Sulfate_DL_PM1_AMS', 'Nitrate_PM1_AMS', 'Nitrate_prec_PM1_AMS', 'Nitrate_DL_PM1_AMS', 'Ammonium_PM1_AMS', 'Ammonium_prec_PM1_AMS', 'Ammonium_DL_PM1_AMS', 'Chloride_PM1_AMS', 'Chloride_prec_PM1_AMS', 'Chloride_DL_PM1_AMS', 'Seasalt_PM1_AMS', 'Seasalt_prec_PM1_AMS', 'Seasalt_DL_PM1_AMS', 'MSA_PM1_AMS', 'MSA_prec_PM1_AMS', 'MSA_DL_PM1_AMS', 'StdtoVol_AMS', 'CloudFlag_AMS', 'AmmBalance_PM1_AMS', 'Density_PM1_AMS', 'OADensity_PM1_AMS', 'OtoC_Ratio_PM1_AMS', 'HtoC_Ra

In [19]:
import os
import icartt
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd 
import numpy as np 


###Convert from mixing ratio to number concentration 
def number_conc(P, T, pbv):
    ###First need to calculate number concentration in air and then 
    ###multiply by the mixing ratio 
    Av = 6.022 * (10**23)
    R = 8.314 

    ##Convert to /cm^3 

    nc = P*Av/(R*T) * (1*10**-6) * pbv

    return(nc) 

###parsing helping function, not using it much 
def arithmetic_split(line):

    plus = line.split("+")

    tokens = []
    ops = [] 

    for item in plus: 
        minus = item.split("-") 

        while len(minus) > 1: 
            curr = minus.pop()
            tokens.append(curr) 
            ops.append("-")

        tokens.append(minus[0].strip())
        ops.append("+")
    
    return(tokens, ops)




import re
regxr = r'\((\S{1,})\)'
regxr = re.compile(regxr)

###Internal function to parse the rate constant expressions 
def parse_rate_constants(infile, temperature=270, pressure=10000):

    rate_dict = {}

    input_csv = pd.read_csv(infile, header=None)
    for (i, input_row) in input_csv.iterrows():
        ###Split line by arithmetic input and then iterate through 
        species = input_row[0]
        rate = input_row[1]
        ##Input temperature 
        rate = rate.replace("T", str(temperature))
        species = species.strip("'")    

        currk = 1 
        tokens = rate.split("*")

        if species == "Acetone_TOGA": 
            currk =  8.8 * (10 ** -12) * np.exp(-1320/temperature) + (1.7 * (10**-14) *np.exp(423/temperature))

        elif species == "CO_NOAA": 
            currk = 1.44*(10**-13)*(1+((number_conc(pressure, temperature, 10**9))/(4.2*(10**19))))
        else:    
            for i, token in enumerate(tokens): 
                currval = 1

                for term in token.split("*"):
                    if "^" in term:    
                        terms = term.replace(")","").replace("(","").split("^")

                        currval = currval * (float(terms[0]) ** float(terms[1]))
            
                    elif "exp" in term.lower():

                        try: 
                            exponent = re.search(regxr, term).groups()[0]
                            currval = currval * np.exp(eval(exponent))
                        except:
                            currval = np.nan
                    else:
                        currval = currval * float(term)
                    
                currk = currk*currval 

        rate_dict[species] = currk

    return rate_dict 




In [20]:
###Place icart files in the following directory 
parent_dir = "./icart_data/"
###List the icart files 
file_list = os.listdir(parent_dir)

In [25]:
###Path to comma-delimited rate constant file. 
###It's probably a pretty brittle function, so more complex rate constants 
###may need to be hardcoded into the parse_rate_contsants function. 

###The file format, per line, should be 
###SPECIES_NAME, RATE_CONSTANT_EXPRESSION
###
###It's important that the SPECIES NAME is the same as the name of the species in the icartt files 
rate_info_csv_path = "voc_rate_constants.csv"

In [26]:
###Instantiate dictionary to be returned and grab a sample of rate constants so we know 
###which species we're dealing with 
base_rates = parse_rate_constants(rate_info_csv_path, float(270)) 
reactivity_dict = {} 
reactivity_dict['lat'] = []
reactivity_dict['lon'] = []
reactivity_dict['pressure'] = []
reactivity_dict['temp'] = [] 
reactivity_dict['time'] = []

for species in base_rates.keys():
    reactivity_dict[species] = []


###Calculate rate constants for each icartt file in the directory 
###Should take 30 seconds or so
for file in file_list:  
    all_data = icartt.Dataset(parent_dir + file).data[:]

    curry = int(file.split("_")[2][:4])
    currm = int(file.split("_")[2][4:6].strip("0"))
    currd = int(file.split("_")[2][6:8].strip("0"))

    currdate=datetime(year=curry, month=currm, day=currd)

    for (i, (lat, lon, t, pres, utc))in enumerate(zip(all_data['Latitude'], all_data['Longitude'], all_data['T'], all_data['P'], all_data['UTC_Start'])):

        currdate = datetime(year=curry, month=currm, day=currd)

        if (np.nan in ([lat, lon, t, pres])):
            continue 
        
        pres = 100 * pres
        currdate = currdate + dt.timedelta(seconds=int(utc))

        rates = parse_rate_constants("voc_rate_constants.csv", float(t), float(pres))
        curr_row = all_data[i]
        
        reactivity_dict['lat'].append(lat)
        reactivity_dict['lon'].append(lon)
        reactivity_dict['pressure'].append(pres)
        reactivity_dict['temp'].append(t)
        reactivity_dict['time'].append(currdate)

        for species in rates.keys(): 
            curr_conc = curr_row[species]
            units = dataset.variables[species].units

            if (units == 'ppt') or (units == 'pptv'): 
                curr_conc = number_conc(pres, t, curr_conc * (10**-12))
            elif (units == 'ppb') or (units == 'ppbv'):
                curr_conc = number_conc(pres, t, curr_conc * (10**-9))
            elif (units == 'ppm') or (units == 'ppbmv'):
                curr_conc = number_conc(pres, t, curr_conc * (10**-6))
            else: 
                raise Exception(f'UNKONWN UNITS: {units}')

            if (curr_conc == np.nan) or (curr_conc < 0):
                reactivity_dict[species].append(np.nan)
            else: 
                curr_conc = rates[species] * curr_conc
                reactivity_dict[species].append(curr_conc)



/Users/ischluesche/miniconda3/envs/chemistry/lib/python3.13/site-packages/icartt/dataset.py:392: UserWarning: Variable short name j23Butanedione_NoProductsSpecified_CAFS does not comply with ICARTT standard v2
  warnings.warn(
/Users/ischluesche/miniconda3/envs/chemistry/lib/python3.13/site-packages/icartt/dataset.py:392: UserWarning: Variable short name C4-C5Alkanes_GMI does not comply with ICARTT standard v2
  warnings.warn(
/Users/ischluesche/miniconda3/envs/chemistry/lib/python3.13/site-packages/icartt/dataset.py:392: UserWarning: Variable short name Days_Since_Land_BdyInf_Most_Recent does not comply with ICARTT standard v2
  warnings.warn(
/Users/ischluesche/miniconda3/envs/chemistry/lib/python3.13/site-packages/icartt/dataset.py:392: UserWarning: Variable short name CO.X does not comply with ICARTT standard v2
  warnings.warn(
/Users/ischluesche/miniconda3/envs/chemistry/lib/python3.13/site-packages/icartt/dataset.py:392: UserWarning: Variable short name Flag.CO.X does not comply

In [23]:
currf = pd.DataFrame(reactivity_dict)

In [24]:
currf

,lat,lon,pressure,temp,time,O3_GMI,CO_NOAA,CH3Cl_TOGA,CH2Cl2_TOGA,CHCl3_TOGA,...,MAC_TOGA,Acetone_TOGA,MEK_TOGA,CH3OH_TOGA,C2H5OH_TOGA,MTBE_TOGA,DMS_TOGA,HCN_TOGA,CH3CN_TOGA,EthONO2_TOGA
0,61.245643,-150.123636,95035.583333,284.955833,2016-08-03 19:02:05,0.016250,NaN,NaN,0.000162,0.000030,...,0.012319,1.401336e+10,0.006891,0.021500,0.021238,NaN,NaN,0.000076,0.000037,NaN
1,61.240264,-150.427757,79790.111111,277.962778,2016-08-03 19:04:05,0.018664,1.212450e+08,NaN,0.000123,0.000021,...,0.005026,1.085543e+10,0.004852,0.015210,0.009076,NaN,NaN,0.000071,0.000027,NaN
2,61.144666,-150.594062,68924.472222,272.466389,2016-08-03 19:06:05,0.044823,1.321525e+08,NaN,0.000110,0.000017,...,NaN,1.846569e+10,0.004648,0.021109,NaN,NaN,NaN,0.000177,0.000057,NaN
3,60.954164,-150.618058,61761.166667,268.874167,2016-08-03 19:08:05,0.045999,1.333712e+08,NaN,0.000090,0.000015,...,0.002024,2.381708e+10,0.002885,0.026246,NaN,NaN,NaN,0.000302,0.000086,NaN
4,60.799499,-150.871766,54267.666667,261.296111,2016-08-03 19:10:05,0.040385,1.946552e+08,NaN,0.000068,0.000013,...,NaN,3.272037e+10,0.003311,0.039030,0.003074,NaN,NaN,0.000507,0.000118,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12163,-14.229906,-170.654848,70229.694444,285.413889,2016-08-07 02:24:04,0.031338,6.419430e+07,NaN,0.000063,0.000010,...,NaN,4.400454e+09,0.004478,0.011950,NaN,NaN,NaN,0.000051,0.000055,NaN
12164,-14.358253,-170.732428,76307.861111,287.152778,2016-08-07 02:26:05,0.031440,7.938675e+07,NaN,0.000062,0.000011,...,0.001619,5.712825e+09,0.005946,0.015971,0.003554,NaN,NaN,0.000044,0.000066,NaN
12165,-14.410297,-170.834786,85458.472222,291.708611,2016-08-07 02:28:05,0.029592,8.631081e+07,NaN,0.000078,0.000013,...,0.001561,6.249164e+09,0.008752,0.020238,0.005454,NaN,NaN,0.000049,0.000075,NaN
12166,-14.406830,-170.872844,93139.000000,295.740000,2016-08-07 02:30:04,0.029702,1.062190e+08,NaN,0.000083,0.000014,...,0.001781,7.069258e+09,0.008165,0.020834,0.005206,NaN,NaN,0.000046,0.000083,NaN
